In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F  
#from DNN_utils import flatten 
from torch.utils.data import Dataset, DataLoader, sampler

import numpy as np 
import data_utils
import pandas as pd

df_train = pd.read_csv('processed_data/dnn_train_never_train.csv')

# Configurations 
df_train = data_utils.df_train # The dataset will be defined data_utils.
NUM_TRAIN = 10000 # Number of training examples for splitting training and validation datasets. 
NUM_ROWS = len(df_train)
device = 'cpu'
dtype = torch.float32
print_every = 100

ImportError: cannot import name 'load_data' from partially initialized module 'data_utils' (most likely due to a circular import) (/Users/eminozyoruk/Library/CloudStorage/GoogleDrive-emin.ozyoruk@chicagobooth.edu/My Drive/Chicago/Academic/Booth 4.3/Probabilistic Graphical Models/Project/Codes/keyword-spotting/data_utils.py)

In [9]:
class DNN_FC(nn.Module):
    def __init__(self, input_size, num_classes):
        super().__init__()
        # assign layer objects to class attributes
        # We may write a loop if we use the same activation function for all layers.
        self.fc1 = nn.Linear(input_size, input_size)
        nn.init.kaiming_normal_(self.fc1.weight)
        self.fc2 = nn.Linear(input_size, input_size)
        nn.init.kaiming_normal_(self.fc2.weight)
        self.fc3 = nn.Linear(input_size, input_size)
        nn.init.kaiming_normal_(self.fc3.weight)
        self.fc4 = nn.Linear(input_size, input_size)
        nn.init.kaiming_normal_(self.fc4.weight)
        self.fc5 = nn.Linear(input_size, num_classes)
        nn.init.kaiming_normal_(self.fc5.weight)
    
    def forward(self, x):
        x_temp = x
        x_temp = F.relu(self.fc1(x_temp))
        x_temp = F.relu(self.fc2(x_temp))
        x_temp = F.relu(self.fc3(x_temp))
        x_temp = F.relu(self.fc4(x_temp))
        scores = self.fc5(x_temp)
        return scores


def test_DNN_FC():
    input_size = 20  # Feature dimension for mfcc
    num_classes = 11 # Number of phoneme classes
    minibatch_size = 64
    dtype = torch.float32
    x = torch.zeros((minibatch_size, input_size), dtype=dtype)  # minibatch size 64, feature dimension 20
    model = DNN_FC(input_size, num_classes)
    scores = model(x)
    print(scores.size())  # you should see [minibatch_size, num_classes]
test_DNN_FC()

torch.Size([64, 11])


In [ ]:
# Convert dataset into a format that torch can read.
class CustomDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.df = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        mfcc = self.df.iloc[idx]['mfcc']
        label = self.df.iloc[idx]['label']
        
        mfcc = torch.tensor(mfcc, dtype=torch.float32)
        label = torch.tensor(label, dtype=torch.long)
        
        if self.transform:
            mfcc = self.transform(mfcc)

        return mfcc, label

# Create an instance of your dataset with your DataFrame
dataset_train = CustomDataset(df_train)  # Assuming df is your pandas DataFrame

# Create the DataLoader to handle batching
loader_train = DataLoader(dataset_train, batch_size=64,
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

loader_val = DataLoader(dataset_train, batch_size=64,
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, NUM_ROWS)))


In [ ]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=dtype) 
            scores = model(x) 
            _, preds = scores.max(1) 
            true_class = y.argmax(dim=1) # True class is the one that has the highest probability in the data.
            num_correct += (preds == true_class).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [ ]:
device = 'cpu'
dtype = torch.float32

def train(model, optimizer, df, epochs=1):
    """
    Train the model using the PyTorch Module API.

    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for

    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                check_accuracy(loader_val, model)
                print()

In [ ]:
input_size = len(df_train['mfcc'][0])
num_classes = len(df_train['label'][0])
learning_rate = 1e-2
model = DNN_FC(input_size, num_classes)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

train(model, optimizer)
